In [1]:
#Import needed packages
import numpy as np
import matplotlib.pylab as py
import matplotlib.pyplot as plt
from qiskit.visualization import *
from qiskit import *
from qiskit.aqua import *
from qiskit.quantum_info import *
from qiskit.quantum_info.states import *
%config InlineBackend.figure_format='retina'
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_textbook.tools import array_to_latex
from qiskit.extensions import RXGate, XGate, CXGate,IGate
from qiskit.providers.aer.extensions import Snapshot
from ising_kitaev import initialize_chain, trotter, run_adiabatic_zeeman_change, rotate_to_measurement_basis, add_measurement,trotter,determine_intermediate_zeemans
from ising_kitaev import move_chain, run_adiabatic_zeeman_change,initialize_chain, braid_chain, rotate_to_measurement_basis, add_measurement, initialize_coupler

In [2]:
#Define Heisenberg interaction terms
def zz_interaction(qc, q, i0, i1, coup):
    qc.cx(q[i0],q[i1])
    qc.u1(-coup,q[i1])
    qc.cx(q[i0],q[i1])

def xx_interaction(qc, q, i0, i1, coup):
    qc.h(q[i0])
    qc.h(q[i1])
    qc.cx(q[i0],q[i1])
    qc.u1(-coup,q[i1])
    qc.cx(q[i0],q[i1])
    qc.h(q[i0])
    qc.h(q[i1])

#Define Zeeman interaction term
def zeeman_term(qc, q, i, h):
    qc.ry(h[i], q[i])

#Define the total hamiltonian with the Heiseneberg and the zeeman interactions
def chain_hamiltonian(qc, q, J1, h,J2, debug=False):
    n=len(q)
    for j in range(0, n, 2):
        zz_interaction(qc, q, j, j+1, J1)
        xx_interaction(qc, q, j, j+1, J1)
    for j in range(1, n-1, 2):
        zz_interaction(qc, q, j, j+1, J1)
        xx_interaction(qc, q, j, j+1, J1)
        if(debug):
            qc.barrier()
    for j in range(n):
        zeeman_term(qc, q, j, h)

    if(debug):
        qc.barrier()

#Define a single trotter step
def trotter_step(qc, q, J1, h,J2):
    chain_hamiltonian(qc, q, J1, h, J2)

#Perform n trotter steps
def trotter(qc, q, h, dt, J2, nsteps, debug=False):
    for i in range(nsteps):
        trotter_step(qc, q, dt, h*dt,J2)

In [3]:
#Estimate the zeeman gap
def estimate_gap(zeeman):
    return 2*np.abs(np.min(zeeman)) - 1.0

#Find the intermediate zeemans that will be used when ramping down the zeeman interaction
def find_int_zeemans(iz,fz,steps):
    zeemans=[iz.copy() for i in range(steps)]
    for i in range(len(iz)):
        if iz[i] == fz[i]:
            for j in range(steps):
                zeemans[j][i] = iz[i]
        else:
            for j, v in enumerate(np.linspace(iz[i],fz[i],steps)):
                zeemans[j][i] = v
    return zeemans

#Apply the adiabatic zeeman changes with the Trotter approximation 
def adiabatic_z_change(circuit, qreg, initial_zeeman, final_zeeman,J2,  gap_fraction, min_increment,
                                delay, trotter_step_number):

    zi = np.copy(initial_zeeman)
    zeeman_diff = final_zeeman - initial_zeeman
    
    zeeman_distance = np.max(np.abs(zeeman_diff))
    zeeman_update_sign = np.sign(zeeman_diff)
    dt = delay / trotter_step_number
    
    # First evolve the system.
    trotter(circuit, qreg, initial_zeeman, dt, J2,
            trotter_step_number)
    
    # Evolve the system till we reach the final zeeman.
    i = 0
    while zeeman_distance > 0:
        gap = estimate_gap(zi)
        zeeman_step = min(max(gap_fraction*gap, min_increment),
                          zeeman_distance)
        zi += zeeman_update_sign*zeeman_step
        trotter(circuit, qreg, zi, dt, J2, trotter_step_number)

        zeeman_distance -= zeeman_step

In [ ]:
L=4 #Chain size
N=L*3 #Total number of qubits: resvoir + chain + resvoir

J1=1.0 #Normalized neareast-neighbor interaction strength
J2=0.0
iz=10.0 #Initial zeeman strength on chain
izeeman=np.array([0.0]*L+[10.0]*L+[0.0]*L)
fz=0.00 #Final zeeman strength on chain
fzeeman=np.array([0.0]*N)

#Creat circuit and registers
q=QuantumRegister(N)
c=ClassicalRegister(L)
qc=QuantumCircuit(q,c)

#Find the intermediate zeeman configurations between the initial and final configurations
zeemans=find_int_zeemans(izeeman,fzeeman,11)

#Initialize the chain in the appropriate states
for i in range(4, N-4):
    qc.h(q[i])
    qc.s(q[i])

for i in range(0, N-9, 2):
    qc.h(q[i])
    qc.cx(q[i],q[i+1])
    qc.z(q[i])
    qc.x(q[i+1])
for i in range(8, N-1, 2):
    qc.h(q[i])
    qc.cx(q[i],q[i+1])
    qc.z(q[i])
    qc.x(q[i+1])

initial_zeeman = izeeman

#Apply the adiabatic evolution of the system 
for zeeman in zeemans:
    adiabatic_z_change(qc, q, initial_zeeman, zeeman,J2, 0.25, 0.25, 2, 20)
    initial_zeeman=zeeman

#Perform the appropriate rotation for the measurement scheme
for i in range(N):
    qc.rx(-np.pi/2, q[i])

for i in range(4,N-4,2):
    qc.cx(q[i],q[i+1])    
    qc.h(q[i])
    
qc.measure(q[4:8],c) 
val_counts=execute(qc,Aer.get_backend('qasm_simulator')).result().get_counts()
plot_histogram(val_counts,color='midnightblue', title="Singlet-Triplet Profile")